# Costos y Beneficios asociados a la implementación del SMI por Comuna (Análisis instantáneo)

Primero se realiza el cálculo de los costos de implementación de SMI en Chile y luego se estiman los beneficios. Para esto se toma como base la información proveniente de la tabla *df_caracterizacion_distribucion*, la cual se ha obtenido a partir de su procesamiento en "*Caracterizacion_Dx_Dic2024_comuna (CNE).ipynb*".

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from pprint import pprint

In [2]:
df_caracterizacion_distribucion = pd.read_excel("./Para obtener la caracterización de Dx/Salidas/Caracterizacion_Dx_Dic2024_comuna (CNE).xlsx")

# Dolar y UF considerados (a 16 de Oct de 2025, según SII)
usd = 959.36
uf 	= 39_521.26

# Se elimina la columna "Cantidad de alimentadores" que no es necesaria para el análisis (no incluidas en el SMMC de la Norma)
df_caracterizacion_distribucion = df_caracterizacion_distribucion.drop(columns=["Cantidad de alimentadores",
                                                                                "Energía facturada [USD]"])

# Se setea la comuna y el segmento como índice del DataFrame
df_caracterizacion_distribucion = df_caracterizacion_distribucion.set_index(["Comuna", "Segmento"], drop=False)

# Se anualiza la Energía facturada
df_caracterizacion_distribucion["Energía promedio mensual [kWh]"] = (df_caracterizacion_distribucion["Energía promedio mensual [kWh]"] * 12) / 1000  # Conversión a MWh/año

# Se renombran columnas para mayor claridad
df_caracterizacion_distribucion = df_caracterizacion_distribucion.rename(columns={
	"Energía promedio mensual [kWh]": "Energía facturada anual [MWh/año]"
})

In [ ]:
# Información al mes de diciembre de 2024
# -> Cantidad de clientes.
# -> Cantidad de energía.
cantidad_clientes = df_caracterizacion_distribucion['Cantidad de clientes'].sum()
cantidad_energia = df_caracterizacion_distribucion['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'Residencial']
cantidad_clientes_residenciales = df_facturacion_clientes_residenciales['Cantidad de clientes'].sum()
cantidad_energia_residenciales = df_facturacion_clientes_residenciales['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'No Residencial BT']
cantidad_clientes_no_residenciales_bt = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_bt = df_facturacion_clientes_no_residenciales['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'No Residencial AT']
cantidad_clientes_no_residenciales_at = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_at = df_facturacion_clientes_no_residenciales['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'LibreDx']
cantidad_clientes_no_residenciales_libredx = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_libredx = df_facturacion_clientes_no_residenciales['Energía facturada anual [MWh/año]'].sum()

cantidad_um_1f_td = df_caracterizacion_distribucion['UM 1F para TD'].sum()
cantidad_um_3f1s_td = df_caracterizacion_distribucion['UM 3F 1S para TD'].sum()
cantidad_um_3f2s_td = df_caracterizacion_distribucion['UM 3F 2S para TD'].sum()

print("(A fecha de diciembre de 2024)",
      "\nClientes regulados:", 
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes), 
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia / 1_000).round(2)),
      "\nClientes Residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_residenciales / 1_000).round(2)),
      "\nClientes No Residenciales en BT:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_bt),
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_no_residenciales_bt / 1_000).round(2)),
      "\nClientes No Residenciales en AT:",
	  "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_at),
	  "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_no_residenciales_at / 1_000).round(2)),
      "\nClientes No Residenciales LibreDx:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_libredx),
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_no_residenciales_libredx / 1_000).round(2)),
      "\n--------------------------------------------",
      "\nCantidad de UM para TD:",
	  "\n -> UM 1F para TD: {:,}".format(cantidad_um_1f_td),
	  "\n -> UM 3F 1S para TD: {:,}".format(cantidad_um_3f1s_td),
	  "\n -> UM 3F 2S para TD: {:,}".format(cantidad_um_3f2s_td),
	  "\n")

## Definición del tipo de tecnología a utilizar en función de la densidad

In [3]:
# Decisión de la tecnología a ocupar; PLC, Celular, RF, LoRa, etc.
df_implementacion = df_caracterizacion_distribucion.copy()

# Se definen los niveles de densidad que existen en la tabla
niveles_densidad = [
    "EXTREMADAMENTE BAJA",
    "MUY BAJA",
    "BAJA",
    "MEDIA",
    "ALTA"
]

print("Se asigna la tecnología de comunicaciones para cada nivel de densidad")
print("Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'")
print("----------------------------------------------------")

# Input de tecnología y armando del mapeo
mapeo_tecnologia = {}

for densidad in niveles_densidad:
    tecnologia = input(f"Tecnología para una densidad '{densidad}': ")
    if tecnologia not in ['Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa']:
        while True:
            print("Inválida. Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'.")
            tecnologia = input(f"Tecnología para una densidad '{densidad}': ")
            if tecnologia in ['Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa']:
                break
    mapeo_tecnologia[densidad] = tecnologia
    
print("Tecnologías asignadas por nivel de densidad:")
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")

# Se crea la nueva columna usando la columna 'Densidad'
df_implementacion["Tecnología"] = df_implementacion["Densidad"].map(mapeo_tecnologia)

Se asigna la tecnología de comunicaciones para cada nivel de densidad
Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'
----------------------------------------------------
Tecnologías asignadas por nivel de densidad:
 - EXTREMADAMENTE BAJA: TWACS
 - MUY BAJA: TWACS
 - BAJA: TWACS
 - MEDIA: TWACS
 - ALTA: TWACS


## Cálculo sobre la cantidad de equipos necesarios por comuna y segmento

Supuestos considerados para asignar las UM necesarias en cada segmento:

 - Cliente Residencial + No Residencial BT + TD 1F ....................... -> Se le asigna la Unidad de Medida (UM) 1F.
 - Cliente No Residencial AT ........................... -> Se le asigna la Unidad de Medida (UM) 3F de conexión directa (sin CT).
 - TD 3F 1S + TD 3F 2S + Cliente libre ............-> Se le asigna la Unidad de Medida (UM) 3F de conexión indirecta (con CT).

### Requerimientos de Unidades de Medida (UM)

In [ ]:
# Se crea el df asociado a una implementación real temporal
crecimiento_anual_demanda = 0.0302  			# Crecimiento anual de la demanda (3.02%)

crecimiento_anual_residencial = 0.0153  		# Crecimiento anual de clientes residenciales (BT) (1.53%)
crecimiento_anual_no_residencial = 0.0094  		# Crecimiento anual de clientes no residenciales (AT) (0.94%)
crecimiento_anual_libredx = 0.0315  			# Crecimiento anual de clientes libre Dx (3.15%)

df_implementacion_1 = df_implementacion.copy()		# Año 1 (2026)
df_implementacion_2 = df_implementacion.copy()		# Año 2 (2027)
df_implementacion_3 = df_implementacion.copy()		# Año 3 (2028)
df_implementacion_4 = df_implementacion.copy()		# Año 4 (2029)
df_implementacion_5 = df_implementacion.copy()		# Año 5 (2030)
df_implementacion_6 = df_implementacion.copy()		# Año 6 (2031)
df_implementacion_7 = df_implementacion.copy()		# Año 7 (2032)
df_implementacion_8 = df_implementacion.copy()		# Año 8 (2033)
df_implementacion_9 = df_implementacion.copy()		# Año 9 (2034)
df_implementacion_10 = df_implementacion.copy()		# Año 10 (2035)
df_implementacion_11 = df_implementacion.copy()		# Año 11 (2036)
df_implementacion_12 = df_implementacion.copy()		# Año 12 (2037)
df_implementacion_13 = df_implementacion.copy()		# Año 13 (2038)
df_implementacion_14 = df_implementacion.copy()		# Año 14 (2039)
df_implementacion_15 = df_implementacion.copy()		# Año 15 (2040)

lista_dfs_implementacion = [df_implementacion,
							df_implementacion_1,
							df_implementacion_2,
							df_implementacion_3,
							df_implementacion_4,
							df_implementacion_5,
							df_implementacion_6,
							df_implementacion_7,
							df_implementacion_8,
							df_implementacion_9,
							df_implementacion_10,
							df_implementacion_11,
							df_implementacion_12,
							df_implementacion_13,
							df_implementacion_14,
							df_implementacion_15]

# Los datos utilizados de energía y cantidad de clientes corresponden al año 2024 (df_implementacion), sin embargo,
# el año base de la implementación es 2025 (df_implementacion_0), salvo para el caso de lo clientes libres el cual si está actualizado a 2025.
factor_crecimiento_cantidad_clientes = np.where(df_implementacion["Segmento"] == 'Residencial', crecimiento_anual_residencial,
											np.where(df_implementacion["Segmento"] == 'No Residencial BT', crecimiento_anual_residencial,
												np.where(df_implementacion["Segmento"] == 'No Residencial AT', crecimiento_anual_no_residencial, 0)))

factor_crecimiento_demanda = np.where(df_implementacion["Segmento"] == 'Residencial', crecimiento_anual_demanda,
								np.where(df_implementacion["Segmento"] == 'No Residencial BT', crecimiento_anual_demanda,
									np.where(df_implementacion["Segmento"] == 'No Residencial AT', crecimiento_anual_demanda, 0)))

# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado (factor compuesto)
lista_dfs_implementacion[0]["Cantidad de clientes"] = ((lista_dfs_implementacion[0]["Cantidad de clientes"]
														* (1 + factor_crecimiento_cantidad_clientes)).round().astype('Int64'))

# Se actualiza la Energía facturada anual según el crecimiento anual esperado (factor compuesto)
lista_dfs_implementacion[0]["Energía facturada anual [MWh/año]"] = (lista_dfs_implementacion[0]["Energía facturada anual [MWh/año]"]
																	* (1 + factor_crecimiento_demanda))


for periodo in range(len(lista_dfs_implementacion)):

	# Se agrega la columna de "Cantidad UM 1F" según el segmento
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		
		if row['Segmento'] == 'Residencial' or row['Segmento'] == 'No Residencial BT':
			valores.append(row['Cantidad de clientes'])
		elif row['Segmento'] == 'Red':
			valores.append(row['UM 1F para TD'])
		else:
			valores.append(0)

	lista_dfs_implementacion[periodo]["Cantidad UM 1F"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index).round().astype('Int64')

	# Se agrega la columna de "Cantidad UM 3F Directa" según el segmento
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		
		if row['Segmento'] == 'No Residencial AT':
			valores.append(row['Cantidad de clientes'])
		else:
			valores.append(0)

	lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index).round().astype('Int64')

	# Se agrega la columna de "Cantidad UM 3F Indirecta" según el segmento
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		
		if row['Segmento'] == 'Red' or row['Segmento'] == 'LibreDx':
			valores.append(row['UM 3F 1S para TD'] + row['UM 3F 2S para TD'] * 2 + row['Cantidad de clientes'])
		else:
			valores.append(0)

	lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index).round().astype('Int64')

	# Se preparan los datos para el siguiente período (factor compuesto)
	lista_dfs_implementacion[periodo]["Cantidad de clientes"] = (lista_dfs_implementacion[0]["Cantidad de clientes"]
															  	 * (1 + crecimiento_anual_residencial) ** periodo).round().astype('Int64')

	if periodo + 1 < len(lista_dfs_implementacion):
		lista_dfs_implementacion[periodo + 1]["UM 1F para TD"] = 0
		lista_dfs_implementacion[periodo + 1]["UM 3F 1S para TD"] = 0
		lista_dfs_implementacion[periodo + 1]["UM 3F 2S para TD"] = 0

		# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado
		factor_crecimiento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'Residencial', crecimiento_anual_residencial,
								np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'No Residencial BT', crecimiento_anual_residencial,
				 				np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'No Residencial AT', crecimiento_anual_no_residencial,
								np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'LibreDx', crecimiento_anual_libredx,
								0))))
		
		# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado (factor compuesto)
		lista_dfs_implementacion[periodo + 1]["Cantidad de clientes"] = (((lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																		* (1 + factor_crecimiento)).round().astype('Int64')) 
																		- (lista_dfs_implementacion[periodo]["Cantidad de clientes"]))

		# Se actualiza la Energía facturada anual según el crecimiento anual esperado (factor compuesto)
		lista_dfs_implementacion[periodo + 1]["Energía facturada anual [MWh/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																					* (1 + crecimiento_anual_demanda))

### Requerimientos de Unidades Concentradoras (DCU)

Supuestos considerados para las DCU de RF y LoRa:
 - Tecnología RF-Mesh: Cobertura de 800 metros a la redonda (extendible a través de UM) y soporta 1000 dispositivos.
 - Tecnología LoRa: Cobertura de **2km** a la redonda en zona urbana (zonas Alta y Media) y **15km** en rural (ie, zonas Baja, Muy Baja y Extremadamente Baja).

In [8]:
# Se calclula la cantidad de Unidades Concentradoras (DCU) necesarias
valores = []

for i, row in df_implementacion.iterrows():
    
    if row["Tecnología"] == "G3-PLC" and row["Segmento"] == "Red":
        dcu_requeridas = (row['UM 1F para TD'] + row['UM 3F 1S para TD'] + row['UM 3F 2S para TD'])		# Sujeto a cambios según estimación de TD
        valores.append(dcu_requeridas)
    
    elif row["Tecnología"] == "RF-Mesh" and row["Segmento"] == "Red":
        capacidad = 1000  																								# Capacidad de 1000 equipos por DCU
        cantidad_cliente_comuna = df_implementacion.loc[(row["Comuna"], slice(None)), "Cantidad de clientes"].sum()			
        dcu_requeridas = (cantidad_cliente_comuna // capacidad) + 1  													# Redondeo hacia arriba
        valores.append(dcu_requeridas)

    elif row["Tecnología"] == "LoRa" and row["Segmento"] == "Red":
        if row["Densidad"] == "EXTREMADAMENTE BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "BAJA":
            cobertura_km2 = (15 ** 2) * np.pi 
            dcu_requeridas = np.ceil(row["Superficie efectiva [km2]"] / cobertura_km2)					# Rural
            valores.append(dcu_requeridas) 	
        else:
            cobertura_km2 = (2 ** 2) * np.pi
            dcu_requeridas = np.ceil(row["Superficie efectiva [km2]"] / cobertura_km2)					# Urbano
            valores.append(dcu_requeridas)
        
    elif row["Tecnología"] == "TWACS" and row["Segmento"] == "Red":										# Caso TWACS: 1 DCU por SPD
        dcu_requeridas = row["Cantidad SPD"]							
        valores.append(dcu_requeridas)

    else:        
        dcu_necesarias = 0								# Caso de Celular y otros segmentos
        valores.append(dcu_necesarias)

df_implementacion["Cantidad DCU"] = pd.Series(valores, index=df_implementacion.index).astype('Int64')

# Se asignan 0 DCU para el resto de los años
for periodo in range(1, len(lista_dfs_implementacion)):
	lista_dfs_implementacion[periodo]["Cantidad DCU"] = 0

### Requerimientos de comunicaciones para el tramo DCU -> SGO

Se calcula la cantidad de datos requeridos a transferir entre el DCU y el SGO, considerando mediciones, alarmas, actualizaciones y control (Estudio del [Anexo 4-1, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/:x:/r/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/Doc.aspx?sourcedoc=%7B8873AC9E-7FED-4D32-95FF-28A4F39C3069%7D&file=Anexo%204-10%20-CNE-SM-COSTOS%20ITP_final.xlsx&action=default&mobileredirect=true)). Estas cantidades dependen del tipo de tecnología a utilizar. Además, se han utilizado precios provenientes de una cotización realizada con Entel para el mismo proceso VAD.

Se ha considerado (Basado en el cálculo del mismo Anexo 4-1 VAD 2024-2028):
- 110.2 - kB por cliente trifásico en zona de ALTA densidad de clientes.
- 45.7 - kB por cliente trifásico en zona de BAJA densidad de clientes.
- 71.7 - kB por cliente monofásico en zona de ALTA densidad de clientes.
- 32.5 - kB por cliente monofásico en zona de BAJA densidad de clientes.
- 110.2 - kB por trafo trifásico en zona de ALTA densidad de clientes.
- 45.7 - kB por trafo trifásico en zona de BAJA densidad de clientes.
- 83.7 - kB por trafo monofásico en zona de ALTA densidad de clientes.
- 38.5 - kB por trafo monofásico en zona de BAJA densidad de clientes.

In [ ]:
# Se calcula la cantidad de datos transmitida por DCU (Cálulos realizados en 'Costos.xlsx')

# 1: Se consideran las exigencias de medición de la Norma SMMC hasta antes del 2034 y después de ella (Artículo 9-8)
# 0: Se consideran las exigencias de medición relajadas para que TWACS pueda operar
while True:
	try:
		exigencias_de_medicion = float(input("Se aplica la exigencia de la NTD? (1 = Sí, 0 = No): "))
		if exigencias_de_medicion == 1 or exigencias_de_medicion == 0:
			break
	except ValueError:
		pass

if exigencias_de_medicion == 1:
	datos_cliente_3f_zona_alta = 110.2 / 1024					# MB mensual por cliente
	datos_cliente_3f_zona_baja_previo_2034 = 45.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024					# MB mensual por cliente
	datos_cliente_1f_zona_baja_previo_2034 = 32.5 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024						# MB mensual por trafo
	datos_trafo_3f_zona_baja_previo_2034 = 45.7 / 1024			# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024						# MB mensual por trafo
	datos_trafo_1f_zona_baja_previo_2034 = 38.5 / 1024			# MB mensual por trafo

	# Valores que cambian después del 2034
	datos_cliente_3f_zona_baja_post_2034 = 110.2 / 1024			# MB mensual por cliente
	datos_cliente_1f_zona_baja_post_2034 = 71.7 / 1024			# MB mensual por cliente

	datos_trafo_3f_zona_baja_post_2034 = 110.2 / 1024			# MB mensual por trafo
	datos_trafo_1f_zona_baja_post_2034 = 83.7 / 1024			# MB mensual por trafo

else:	# Se consideran las exigencias de medición relajadas para que TWACS pueda operar
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 30.3 / 1024		# MB mensual por cliente (modificado para TWACS) -> cada 6 horas = 32,3, cada 12 horas = 30,9, cada 24 horas = 30,3
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 27.4 / 1024		# MB mensual por cliente (modificado para TWACS) -> cada 6 horas = 28,1, cada 12 horas = 27,6, cada 24 horas = 27,4

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 45.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 38.5 / 1024		# MB mensual por trafo

for periodo in range(len(lista_dfs_implementacion)):
	valores = []

	if periodo < 9:		# Hasta antes del 2034

		for i, row in lista_dfs_implementacion[periodo].iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
			cantidad_dcu = lista_dfs_implementacion[0].loc[i, "Cantidad DCU"]

			# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
			if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA"):
				total_clientes_3f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_alta = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
									+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
				valores.append(mb_necesarios)

			elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA"):
				total_clientes_3f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_baja = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_baja_previo_2034 * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja_previo_2034 * total_clientes_1f_zona_baja
									+ datos_trafo_3f_zona_baja_previo_2034 * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja_previo_2034 * total_trafo_1f_zona_baja) / cantidad_dcu
				valores.append(mb_necesarios)

			else:        
				mb_necesarios = 0								# Caso de Celular, TWACS y otros segmentos
				valores.append(mb_necesarios)

		lista_dfs_implementacion[periodo]["MB Mensuales por DCU"] = 0.0
		filtro = lista_dfs_implementacion[periodo]["Segmento"] == "Red"
		
		lista_dfs_implementacion[periodo].loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index[filtro]).round(4)


	else:		# Después del 2034

		for i, row in lista_dfs_implementacion[periodo].iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
			cantidad_dcu = lista_dfs_implementacion[0].loc[i, "Cantidad DCU"]

			# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
			if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA"):
				total_clientes_3f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_alta = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
									+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
				valores.append(mb_necesarios)

			elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA"):
				total_clientes_3f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_baja = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_baja_post_2034 * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja_post_2034 * total_clientes_1f_zona_baja
									+ datos_trafo_3f_zona_baja_post_2034 * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja_post_2034 * total_trafo_1f_zona_baja) / cantidad_dcu
				valores.append(mb_necesarios)

			else:        
				mb_necesarios = 0								# Caso de Celular, TWACS y otros segmentos
				valores.append(mb_necesarios)				

		lista_dfs_implementacion[periodo]["MB Mensuales por DCU"] = 0.0
		filtro = lista_dfs_implementacion[periodo]["Segmento"] == "Red"
		
		lista_dfs_implementacion[periodo].loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index[filtro]).round(4)

In [6]:
# Se calcula la cantidad de datos transmitida por DCU (Cálulos realizados en 'Costos.xlsx')
while True:
	try:
		exigencias_de_medicion = float(input("Se aplica la exigencia de la NTD actual (previa a 2031)? (1 = Sí, 0 = No): "))
		if exigencias_de_medicion == 1 or exigencias_de_medicion == 0:
			break
	except ValueError:
		pass

if exigencias_de_medicion == 1:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 45.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 32.5 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 45.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 38.5 / 1024		# MB mensual por trafo

else:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 71.7 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 83.7 / 1024		# MB mensual por trafo

for periodo in range(len(lista_dfs_implementacion)):
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
		cantidad_dcu = lista_dfs_implementacion[0].loc[i, "Cantidad DCU"]

		# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
		if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA"):
			total_clientes_3f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
			total_clientes_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

			total_trafo_3f_zona_alta = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
							   			+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
			total_trafo_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

			mb_necesarios = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
								 + datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
			valores.append(mb_necesarios)

		elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA"):
			total_clientes_3f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
			total_clientes_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

			total_trafo_3f_zona_baja = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
							   			+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
			total_trafo_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

			mb_necesarios = (datos_cliente_3f_zona_baja * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja * total_clientes_1f_zona_baja
								 + datos_trafo_3f_zona_baja * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja * total_trafo_1f_zona_baja) / cantidad_dcu
			valores.append(mb_necesarios)

		else:        
			mb_necesarios = 0								# Caso de Celular y otros segmentos
			valores.append(mb_necesarios)

	lista_dfs_implementacion[periodo]["MB Mensuales por DCU"] = 0.0
	filtro = lista_dfs_implementacion[periodo]["Segmento"] == "Red"
	
	lista_dfs_implementacion[periodo].loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index[filtro]).round(4)

## Cálculo sobre los costos de inversión por comuna y segmento

Para este primer escenario se han considerado precios de lista de los equipos e instalación, sin descuentos por volumen. Posteriormente se tomará en cuenta esto.

In [13]:
df_capex_smi = pd.read_excel("Costos.xlsx", sheet_name="CAPEX", skiprows=4, usecols="B:M")
df_capex_smi = df_capex_smi.set_index(["Tecnología", "Densidad"], drop=False)					   # Se asignan indices

# Cálculo de la inversión
while True:
	try:
		descuento_por_volumen = float(input("Descuento por volumen (inversión inicial) [%]: ")) / 100      # Definicion del descuento por volumen
		if 0 <= descuento_por_volumen <= 1:
			break
	except ValueError:
		pass

for periodo in range(len(lista_dfs_implementacion)):

	inversion_um_1f = []
	inversion_um_3f_dir = []
	inversion_um_3f_ind = []
	inversion_dcu = []
	inversion_sgo = []
	inversion_mdms = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		tecnologia = row["Tecnología"]
		densidad = row["Densidad"]

		costo_um_1f = (df_capex_smi.loc[(tecnologia, densidad), "UM 1F [USD/u]"] * (1 - descuento_por_volumen))
		costo_um_3f_dir = (df_capex_smi.loc[(tecnologia, densidad), "UM 3F Directa [USD/u]"] * (1 - descuento_por_volumen))
		costo_um_3f_ind = (df_capex_smi.loc[(tecnologia, densidad), "UM 3F Indirecta [USD/u]"] * (1 - descuento_por_volumen))
		costo_dcu = (df_capex_smi.loc[(tecnologia, densidad), "DCU [USD/u]"] * (1 - descuento_por_volumen))
		costo_sgo = df_capex_smi.loc[(tecnologia, densidad), "SGO [USD/Medidor]"]
		costo_mdms = df_capex_smi.loc[(tecnologia, densidad), "MDMS [USD/Medidor]"]

		instalacion_um_1f = (df_capex_smi.loc[(tecnologia, densidad), "Instalación UM 1F [USD/u]"] * (1 - descuento_por_volumen))
		instalacion_um_3f_dir = (df_capex_smi.loc[(tecnologia, densidad), "Instalación UM 3F Directa [USD/u]"] * (1 - descuento_por_volumen))
		instalacion_um_3f_ind = (df_capex_smi.loc[(tecnologia, densidad), "Instalación UM 3F Indirecta [USD/u]"] * (1 - descuento_por_volumen))
		instalacion_dcu = (df_capex_smi.loc[(tecnologia, densidad), "Instalación DCU [USD/u]"] * (1 - descuento_por_volumen))

		inv_um_1f = (row["Cantidad UM 1F"] * costo_um_1f + row["Cantidad UM 1F"] * instalacion_um_1f)
		inv_um_3f_dir = (row["Cantidad UM 3F Directa"] * costo_um_3f_dir + row["Cantidad UM 3F Directa"] * instalacion_um_3f_dir)
		inv_um_3f_ind = (row["Cantidad UM 3F Indirecta"] * costo_um_3f_ind + row["Cantidad UM 3F Indirecta"] * instalacion_um_3f_ind)
		inv_dcu = (row["Cantidad DCU"] * costo_dcu + row["Cantidad DCU"] * instalacion_dcu)
		inv_sgo = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * costo_sgo)
		inv_mdms = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * costo_mdms)

		inversion_um_1f.append(inv_um_1f)
		inversion_um_3f_dir.append(inv_um_3f_dir)
		inversion_um_3f_ind.append(inv_um_3f_ind)
		inversion_dcu.append(inv_dcu)
		inversion_sgo.append(inv_sgo)
		inversion_mdms.append(inv_mdms)

	# Se agregan las columnas de inversión por equipo al DataFrame
	lista_dfs_implementacion[periodo]["Inversión UM 1F [USD]"] = pd.Series(inversion_um_1f, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión UM 3F Directa [USD]"] = pd.Series(inversion_um_3f_dir, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión UM 3F Indirecta [USD]"] = pd.Series(inversion_um_3f_ind, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión DCU [USD]"] = pd.Series(inversion_dcu, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión SGO [USD]"] = pd.Series(inversion_sgo, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión MDMS [USD]"] = pd.Series(inversion_mdms, index=lista_dfs_implementacion[periodo].index)

	# Se suma la inversión total por comuna y segmento
	lista_dfs_implementacion[periodo]["Inversión Total [USD]"] = (lista_dfs_implementacion[periodo]["Inversión UM 1F [USD]"]
															    	+ lista_dfs_implementacion[periodo]["Inversión UM 3F Directa [USD]"]
																  	+ lista_dfs_implementacion[periodo]["Inversión UM 3F Indirecta [USD]"]
																 	+ lista_dfs_implementacion[periodo]["Inversión DCU [USD]"]
																	+ lista_dfs_implementacion[periodo]["Inversión SGO [USD]"]
																	+ lista_dfs_implementacion[periodo]["Inversión MDMS [USD]"]).round(4)


# Inversión total por periodo
print("Inversión total del proyecto dada la siguiente configuración de tecnologías:")
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")
print("\nConsidera además un descuento por volumen de compra e instalación de UM: {:,.0f}%".format(descuento_por_volumen * 100))
print("----------------------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Inversión total periodo {}: M USD {:,.4f}".format(periodo, lista_dfs_implementacion[periodo]["Inversión Total [USD]"].sum() / 1_000_000))

Inversión total del proyecto dada la siguiente configuración de tecnologías:
 - EXTREMADAMENTE BAJA: RF-Mesh
 - MUY BAJA: RF-Mesh
 - BAJA: RF-Mesh
 - MEDIA: RF-Mesh
 - ALTA: RF-Mesh

Considera además un descuento por volumen de compra e instalación de UM: 20%
----------------------------------------------------------------------------
Inversión total periodo 0: M USD 1,524.3752
Inversión total periodo 1: M USD 20.3615
Inversión total periodo 2: M USD 20.6751
Inversión total periodo 3: M USD 20.9903
Inversión total periodo 4: M USD 21.3110
Inversión total periodo 5: M USD 21.6367
Inversión total periodo 6: M USD 21.9664
Inversión total periodo 7: M USD 22.3028
Inversión total periodo 8: M USD 22.6442
Inversión total periodo 9: M USD 22.9927
Inversión total periodo 10: M USD 23.3444
Inversión total periodo 11: M USD 23.7033
Inversión total periodo 12: M USD 24.0667
Inversión total periodo 13: M USD 24.4360
Inversión total periodo 14: M USD 24.8135
Inversión total periodo 15: M USD 25.191

### Resultado de la inversión inicial de SMI en chile

In [14]:
# Para conocer la inversión de cada item por tecnología
suma_por_columna = df_implementacion.iloc[:, 20:27].sum()

print("Inversión por segmento (en millones de dólares):", 
      "\n ->","\n -> ".join(f"{col[0:-6]} [MUSD]: {val / 1_000_000}" for col, val in suma_por_columna.items()))


# Se crea un nuevo DataFrame para almacenar los resultados finales
df_resultado_final = pd.DataFrame()

inversion = [
	"Energía facturada anual [MWh/año]",
	"Cantidad UM 1F",
	"Cantidad UM 3F Directa",
	"Cantidad UM 3F Indirecta",
	"Cantidad DCU",
	"Inversión UM 1F [USD/equipo]",
	"Inversión UM 3F Directa [USD/equipo]",
	"Inversión UM 3F Indirecta [USD/equipo]",
	"Inversión DCU [USD/equipo]",
	"Inversión SGO [USD]",
	"Inversión MDMS [USD]"
]

df_resultado_final["Concepto"] = inversion

for periodo in range(len(lista_dfs_implementacion)):
	
	# Se agregan las inversiones de cada item por periodo
	nuevo_periodo = [
		lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad UM 1F"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad DCU"].sum(),
		lista_dfs_implementacion[periodo]["Inversión UM 1F [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad UM 1F"].sum() if lista_dfs_implementacion[periodo]["Cantidad UM 1F"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión UM 3F Directa [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"].sum() if lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión UM 3F Indirecta [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"].sum() if lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión DCU [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad DCU"].sum() if lista_dfs_implementacion[periodo]["Cantidad DCU"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión SGO [USD]"].sum(),
		lista_dfs_implementacion[periodo]["Inversión MDMS [USD]"].sum()
	]

	df_resultado_final["Periodo {}".format(periodo)] = nuevo_periodo
	
	lista_dfs_implementacion[periodo].drop(columns=["Inversión UM 1F [USD]", "Inversión UM 3F Directa [USD]", 
													"Inversión UM 3F Indirecta [USD]", "Inversión DCU [USD]", 
													"Inversión SGO [USD]", "Inversión MDMS [USD]"], inplace=True)

Inversión por segmento (en millones de dólares): 
 -> Inversión UM 1F [MUSD]: 1170.0722425306667
 -> Inversión UM 3F Directa [MUSD]: 20.03824802666667
 -> Inversión UM 3F Indirecta [MUSD]: 170.09548734
 -> Inversión DCU [MUSD]: 6.672093693333333
 -> Inversión SGO [MUSD]: 146.48950004399998
 -> Inversión MDMS [MUSD]: 11.007645936842106
 -> Inversión Total [MUSD]: 1524.3752175687


## Cálculo Sobre los costos de operación por comuna y segmento

In [ ]:
df_opex_smi = pd.read_excel("Costos.xlsx", sheet_name="OPEX", skiprows=4, usecols="B:G")
df_opex_smi = df_opex_smi.set_index(["Tecnología", "Densidad"], drop=False)			        # Se asignan indices

# Planes de datos, cotización de entel a Dic 2022 (ajustado por IPC, factor=1.12)
plan_10_mb = (1200 / usd) * 1.12  			# Costo mensual en pesos chilenos para 10 MB
plan_10_mb_extra = (100 / usd) * 1.12  		# Costo mensual en pesos chilenos por cada 1 MB extra sobre el plan base de 10 MB
plan_200_mb = (2800 / usd) * 1.12  			# Costo mensual en pesos chilenos para 200 MB
plan_200_300_mb_extra = (50 / usd) * 1.12  	# Costo mensual en pesos chilenos por cada 1 MB extra sobre el plan base de 200 MB
plan_300_mb = (3300 / usd) * 1.12  			# Costo mensual en pesos chilenos para 300 MB

for periodo in range(len(lista_dfs_implementacion)):

	# Cálculo de los costos de operación
	comunicaciones = []
	comunicaciones_dcu_sgo = []
	sgo = []
	mdms = []
	mantenimiento_equipos = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		tecnologia = row["Tecnología"]
		densidad = row["Densidad"]

		opex_com = df_opex_smi.loc[(tecnologia, densidad), "Comunicaciones [USD/medidor]"] 
		opex_sgo = df_opex_smi.loc[(tecnologia, densidad), "Operación SGO [USD/medidor]"]
		opex_mdms = df_opex_smi.loc[(tecnologia, densidad), "Operación MDMS [USD/medidor]"]
		opex_mantenimiento_equipos = df_opex_smi.loc[(tecnologia, densidad), "Soporte y Mantenimiento Equipos [USD/medidor]"]

		operacion_com = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_com)
		operacion_sgo = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_sgo)
		operacion_mdms = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_mdms)
		operacion_mantenimiento_equipos = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_mantenimiento_equipos)
		
		cantidad_dcu = df_implementacion.loc[i, "Cantidad DCU"]
		mb_mensuales = row["MB Mensuales por DCU"]

		if mb_mensuales < 26:                   # Conveniencia económica de plan 10 MB
			operacion_com_dcu_sgo = ((plan_10_mb + round(mb_mensuales - 10) * plan_10_mb_extra) * cantidad_dcu * 12) if mb_mensuales >= 10 else (plan_10_mb * cantidad_dcu * 12)
			comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)
		if 26 <= mb_mensuales <= 210:			# Conveniencia económica de plan 200 MB
			operacion_com_dcu_sgo = ((plan_200_mb + round(mb_mensuales - 200) * plan_200_300_mb_extra) * cantidad_dcu * 12) if mb_mensuales > 200 else (plan_200_mb * cantidad_dcu * 12)
			comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)
		if 210 < mb_mensuales:
			operacion_com_dcu_sgo = ((plan_300_mb + round(mb_mensuales - 300) * plan_200_300_mb_extra) * cantidad_dcu * 12) if mb_mensuales > 300 else (plan_300_mb * cantidad_dcu * 12)
			comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)

		comunicaciones.append(operacion_com)
		sgo.append(operacion_sgo)
		mdms.append(operacion_mdms)
		mantenimiento_equipos.append(operacion_mantenimiento_equipos)

	# Se agregan las columnas de inversión por equipo al DataFrame
	lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"] = pd.Series(comunicaciones, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Operación Comunicaciones DCU-SGO [USD/año]"] = pd.Series(comunicaciones_dcu_sgo, index=lista_dfs_implementacion[periodo].index).astype("Float64")
	lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"] = pd.Series(sgo, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"] = pd.Series(mdms, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"] = pd.Series(mantenimiento_equipos, index=lista_dfs_implementacion[periodo].index)

	if 0 < periodo <= len(lista_dfs_implementacion):
		lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Operación Comunicaciones [USD/año]"]
		lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Operación SGO [USD/año]"]
		lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Operación MDMS [USD/año]"]
		lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Soporte y Mantenimiento Equipos [USD/año]"]

	# Se suma la inversión total por comuna y segmento
	lista_dfs_implementacion[periodo]["Costos Operacionales Totales [USD/año]"] = (lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Operación Comunicaciones DCU-SGO [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"]).round(4)


# Operacion total 
print("Costos de operación totales del proyecto dada la siguiente configuración de tecnologías:\n")
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")
print("-------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Costos de operación periodo {}: M USD {:,.4f}".format(periodo, lista_dfs_implementacion[periodo]["Costos Operacionales Totales [USD/año]"].sum() / 1_000_000))

Costos de operación totales del proyecto dada la siguiente configuración de tecnologías:

 - EXTREMADAMENTE BAJA: RF-Mesh
 - MUY BAJA: RF-Mesh
 - BAJA: RF-Mesh
 - MEDIA: RF-Mesh
 - ALTA: RF-Mesh
-------------------------------------------
Costos de operación periodo 0: M USD 116.5654
Costos de operación periodo 1: M USD 118.2861
Costos de operación periodo 2: M USD 120.0334
Costos de operación periodo 3: M USD 121.8074
Costos de operación periodo 4: M USD 123.6085
Costos de operación periodo 5: M USD 125.4372
Costos de operación periodo 6: M USD 127.2937
Costos de operación periodo 7: M USD 129.1786
Costos de operación periodo 8: M USD 131.0924
Costos de operación periodo 9: M USD 133.0356
Costos de operación periodo 10: M USD 135.0084
Costos de operación periodo 11: M USD 137.0116
Costos de operación periodo 12: M USD 139.0454
Costos de operación periodo 13: M USD 141.1104
Costos de operación periodo 14: M USD 143.2071
Costos de operación periodo 15: M USD 145.3358


### Resultado de los costos de operación y mantenimiento del SMI en chile

In [22]:
# Para conocer la inversión de cada item por tecnología
suma_por_columna = df_implementacion.iloc[:, 21:27].sum()

print("Inversión por segmento (en millones de dólares):", 
      "\n ->","\n -> ".join(f"{col[0:-10]} [MUSD/año]: {val / 1_000_000}" for col, val in suma_por_columna.items()))

# Se crea un nuevo DataFrame para almacenar los resultados finales de OPEX
df_resultado_final_opex = pd.DataFrame()

operacion = [
	"Operación Comunicaciones [USD/año]",
	"Operación Comunicaciones DCU-SGO [USD/año]",
	"Operación SGO [USD/año]",
	"Operación MDMS [USD/año]",
	"Soporte y Mantenimiento Equipos [USD/año]"
]

df_resultado_final_opex["Concepto"] = operacion

for periodo in range(len(lista_dfs_implementacion)):
	
	# Se agregan los costos operacionales de cada item por periodo
	nuevo_periodo_opex = [
		lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Operación Comunicaciones DCU-SGO [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"].sum()
	]

	df_resultado_final_opex["Periodo {}".format(periodo)] = nuevo_periodo_opex

	lista_dfs_implementacion[periodo].drop(columns=["Operación Comunicaciones [USD/año]", "Operación Comunicaciones DCU-SGO [USD/año]", 
                                          			"Operación SGO [USD/año]", "Operación MDMS [USD/año]", 
                                          			"Soporte y Mantenimiento Equipos [USD/año]"], inplace=True)
	
df_resultado_final = pd.concat([df_resultado_final, df_resultado_final_opex], ignore_index=True)

Inversión por segmento (en millones de dólares): 
 -> Operación Comunicaciones [MUSD/año]: 0.0
 -> Operación Comunicaciones DCU-SGO [MUSD/año]: 0.3104049366244163
 -> Operación SGO [MUSD/año]: 97.29346623082057
 -> Operación MDMS [MUSD/año]: 9.047066556526314
 -> Soporte y Mantenimiento Equipos [MUSD/año]: 9.914443848000001
 -> Costos Operacionales Totales [MUSD/año]: 116.56538157300001


## Cálculo sobre los beneficios esperados asociados a la implementación de SMI

In [ ]:
#### Algunas consideraciones y supuestos para el calculo de los beneficios:

# Precio de la energía (PNP equivalente nacional, Anexos PNP Julio 2025)
precio_energia_usd_mwh = 93.14   		# USD/MWh

# Compensaciones automáticas e instruidas por la SEC por fallas en el suministro eléctrico (Fuente: Informe SEC Dic 2024)
monto_compensado_anual = (8_744_000_000 + 3_961_000_000) / usd			# USD/año Automáticas + Instruidas (2024)
cantidad_de_compensaciones = 3_257_118 + 1_150_658								# N° Automáticas + Instruidas (2024)

# Costo de falla 2024 para el SEN bajo una profundidad de entre 0-5%
costo_de_falla_2024 = 415.60			# USD/MWh

# Cálculo de la energía no suministrada anual (fuente SEC)
energia_no_suministrada_anual = monto_compensado_anual / (2 * costo_de_falla_2024)	 	# MWh/año

### Beneficio 1: Ahorro por efecto de la reducción en la forma de lectura pedestre de la energía

Este corresponde al beneficio más directo y perceptible, pues en virtud de la telemedición es posible ahorrar el 100% de los costos incurridos en medición pedestre de energía (informe GTD).

Para la estimación de los costos se hizo uso de la información proveniente de los anexos del VAD ([Anexo 4-i, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FVAD%2FVAD%202024%2D2028%2F02%20Informe%20T%C3%A9cnico%20Preliminar%2FATD1%2FAnexos%2FAnexo%204%2FAnexo%204%2Di%20Atencion%20al%20Cliente&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)) sobre los costos de medición por punto de medición dependiente del ATD.

Cabe destacar que no se cuenta con proyecciones futuras de estos costos, por lo que se ha considerado que se mantienen a lo largo del periodo de evaluación.

In [ ]:
# Se calcula el ahorro anual por reducción en la lectura pedestre
while True:
    lectura_remota = float(input("Reducción en la lectura pedestre [%]: ")) / 100
    if 0 <= lectura_remota <= 1:
	    break

# Costo de lectura pedestre por medición (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
costos_por_densidad = {
    "ALTA": 0.20416,					# USD/medición
    "MEDIA": 0.2653,					# USD/medición
    "BAJA": 0.4133,					    # USD/medición
    "MUY BAJA": 0.7136,					# USD/medición
    "EXTREMADAMENTE BAJA": 0.8853		# USD/medición	
}

for periodo in range(len(lista_dfs_implementacion)):

	# Costo base por registro (por medición)
	costo_base = lista_dfs_implementacion[periodo]["Densidad"].map(costos_por_densidad)

	# Factor anual según periodicidad: mensual = 12, bimensual = 6
	factor_anual = np.where(lista_dfs_implementacion[periodo]["Tipo de facturación"] == "Mensual", 12,
					np.where(lista_dfs_implementacion[periodo]["Tipo de facturación"] == "Bimensual", 6, np.nan))
	
	# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", 1,
			   				np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", 1, 0)))

	# Nueva columna con costo anual
	lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"] = (costo_base
																				* factor_anual
																				* lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																				* indice_segmento
																				* lectura_remota).round(4)

# Ahorro global anual por efectos de la telemedición
print(f"Considera la reducción de un {lectura_remota * 100}% de lectura pedestre")
print("-----------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Ahorro por reducción de lectura pedestre periodo {}: M USD {:,.4f}"
		  .format(periodo, lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"].sum() / 1_000_000))

### Beneficio 2: Ahorro por efecto de la reducción del corte y reposición del suministro remoto

Este beneficio se calcula a través de costo anual destinado a corte y reposición establecido en el VAD ([Anexo 4-1 VAD 2024-2028](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FVAD%2FVAD%202024%2D2028%2F02%20Informe%20T%C3%A9cnico%20Preliminar%2FATD4%2FAnexos%2FAnexo%204%2FAnexo%204%2D1&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)) asociado al ATD4 (basado en la empresa Saesa). Este monto se divide entre la cantidad de clientes de Saesa, y se obtiene entonces el monto anual por cliente.

El informe de GTD afirma que se puede eliminar los costes de corte y reposición en un 81%.

Cabe dastacar que la proyección de los costos ha sido realizada por el mismo proceso VAD, salvo los periodos 13, 14 y 15, que fueron proyectados con un factor de proyección compuesto. De todas formas, VAD considera costos unitarios iguales para todo el horizonte, asi que no tiene importancia.

In [ ]:
# Ahorros asociados al corte y reposición remotos de suministro
while True:
	cortes_remotos = float(input("Reducción de corte y reposición de suministro pedestre [%]: ")) / 100
	if 0 <= cortes_remotos <= 1:
	    break

# Costo promedio por corte e reposición de suministro (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
corte_y_repo = [0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 
		   		0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534]				# USD/cliente-año por año desde 2025 a 2040

for periodo in range(len(lista_dfs_implementacion)):

	# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", 1,
			   				np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", 1, 0)))

	# Ahorro anual por reducción en cortes y reposición remotos de suministro
	lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"] = (lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																						* corte_y_repo[periodo]
																						* cortes_remotos
																						* indice_segmento)

# Ahorro global anual por efectos de la telemedición
print(f"Considera la reducción de un {cortes_remotos * 100}% en cortes y reposición de suministro")
print("--------------------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Ahorro por reducción en corte y reposición remoto periodo {}: M USD {:,.4f}"
		  .format(periodo, lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"].sum() / 1_000_000))

### Beneficio 3: Ahorro por efecto de la reducción de las pérdidas No técnicas

Tomando en cuenta que las pérdidas (tanto técnicas como No técnicas) están calculadas sobre la energía comprada (por lo tanto agrega a todos los clientes en Dx), se debe seguir la siguiente ecuación para el cálculo de esta energía:

$\text{Energía Comprada [MWh]} = \frac{\text{Energía Vendida [MWh]}}{1 - PT - PH - PC}$   

donde, 
 - PT: Pérdidas técnicas 
 - PH: Pérdidas por hurto
 - PC: Pérdidas Comerciales

En cuanto a la reducción en las pérdidas comerciales, considerando que estas tienen que ver con la pérdida por falta de mantenimiento de medidores (como se detalla  acá [Resolución Exenta VAD 2024-2028](https://www.cne.cl/wp-content/uploads/2023/01/ResEx-CNE-N%C2%B0-29-Aprueba-BT-Corregidas-VAD-24-28_20-01-23.pdf), esta se podría llegar a reducir en torno a un 50% tomando en cuenta que la clase de precisión de los MI es mayor y que en su mayoría cuentan con autodiagnóstico.

Por otro lado, para estimar el ahorro que supondría, se tomará como precio de la energía el PNP equivalente nacional calculado en el proceso PNP Julio 2025 ([Anexos PNP Julio 2025](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FPNP%2F2025%2D02%2F02%20ITD%2F4%2E%2DModelo&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)).

In [ ]:
# Se calcula el ahorro anual por reducción del hurto de energía
while True:
    porcentaje_reduccion_hurto = float(input("Reducción en el hurto de energía [%]: ")) / 100
    porcentaje_reduccion_comercial = float(input("Reducción en las pérdidas comerciales [%]: ")) / 100
    if 0 <= porcentaje_reduccion_hurto <= 1 and 0 <= porcentaje_reduccion_comercial <= 1:
	    break
    
# Se fija el crecimiento anual compuesto de la demanda (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
crecimiento_anual_pt = 0.0000			# % anual de crecimiento de las pérdidas técnicas (0%)
crecimiento_anual_ph = 0.0000			# % anual de crecimiento del hurto de energía (0%)
crecimiento_anual_pc = 0.0000			# % anual de crecimiento de las pérdidas comerciales (0%)

for periodo in range(len(lista_dfs_implementacion)):

	lista_dfs_implementacion[periodo]["Pérdidas de energía [pu]"] = (lista_dfs_implementacion[0]["Pérdidas de energía [pu]"]
																		* (1 + crecimiento_anual_pt) ** periodo)
	lista_dfs_implementacion[periodo]["Hurto de energía [pu]"] = (lista_dfs_implementacion[0]["Hurto de energía [pu]"]
																	* (1 + crecimiento_anual_ph) ** periodo)
	lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"] = (lista_dfs_implementacion[0]["Pérdidas comerciales [pu]"]
																		* (1 + crecimiento_anual_pc) ** periodo)

	# Perdidas técnicas (pt), pérdidas por hurto (ph) y pérdidas comerciales (pc) nominales
	pt = lista_dfs_implementacion[periodo]["Pérdidas de energía [pu]"]
	ph = lista_dfs_implementacion[periodo]["Hurto de energía [pu]"]
	pc = lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"]

	# Cálculo de las pérdidas reducidas
	hurto_reducido = lista_dfs_implementacion[periodo]["Hurto de energía [pu]"] * porcentaje_reduccion_hurto
	comercial_reducido = lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"] * porcentaje_reduccion_comercial

	# Se calcula de esa forma dado que los % están sobre la energía comprada, no la facturada
	lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"] = ((((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																					* hurto_reducido)
																					* precio_energia_usd_mwh)
																			+ (((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																					* comercial_reducido)
																					* precio_energia_usd_mwh))

print(f"Considera la reducción de un {porcentaje_reduccion_hurto * 100}% en el hurto de energía")
print(f"Considera la reducción de un {porcentaje_reduccion_comercial * 100}% en las pérdidas comerciales")
print("---------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Ahorro por reducción de PNT periodo {}: M USD {:,.4f}"
		  .format(periodo, lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"].sum() / 1_000_000))

### Beneficio 4: Ahorro por efecto de la reducción del tiempo de aviso y localización de fallas

Para el cálculo de este beneficio se ha usado el monto compensado neto en 2024 que indica la SEC ([Informe SEC Dic 2024](https://www.sec.cl/sitio-web/wp-content/uploads/2025/02/Informe-SEC-Diciembre-2024.pdf)). A partir de este monto se busca la energía no suministrada en base a la siguiente formula definida por la SEC [Compensaciones Dx](https://www.sec.cl/sitio-web/wp-content/uploads/2019/06/ANEXO_PROCESO_COMPENSACIONES_DX-1.pdf):

$\text{Monto a Compensar [USD/año]} = 2 * \text{Energía No Suministrada [MWh/año]} * \text{Costo de Falla [USD/MWh]}$

Para el costo de falla se considera el costo de falla para el SEN bajo una profundidad de entre 0-5% (según [Costos de falla 2024](https://www.diariooficial.interior.gob.cl/publicaciones/2024/09/03/43941/01/2540114.pdf) y [Costos de falla 2025](https://www.cne.cl/wp-content/uploads/2025/10/Rex-503-comunica-CFCD-y-CFLD.pdf))

En cuanto a la reducción promedio del tiempo de No Suministro, [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf) afirma que se puede estimar entre un 5% y un 35%. Esto coincide con lo informado por GTD que lo establece en un 20%.

In [ ]:
# Cantidad de fallas y duración promedio anual (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
# cantidad_fallas_anual = 56855     # Fallas/año
# duracion_promedio_anual = 5.62    # Horas/falla

# Se calcula el ahorro anual por reducción en la duración promedio de las fallas
while True:
    reduccion_duracion_falla = float(input("Reducción en la duración promedio de la falta de suministro [%]: ")) / 100
    if 0 <= reduccion_duracion_falla <= 1:
	    break

costo_de_falla_2025 = 467.19			# USD/MWh

for periodo in range(len(lista_dfs_implementacion)):

	# Energía no suministrada eliminada por año (MWh/año)
	energia_no_suministrada_eliminada = energia_no_suministrada_anual * reduccion_duracion_falla

	beneficio_neto_compensacion_evitada = energia_no_suministrada_eliminada * costo_de_falla_2025 * 2		# USD/año

	# Se asigna una parte a cada segmento según su participación en las compensaciones totales
	ahorro_por_compensacion_evitada_parcializado = beneficio_neto_compensacion_evitada / ((len(lista_dfs_implementacion[periodo]) / 5) * 3)
	lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"] = np.where(lista_dfs_implementacion[periodo]["Segmento"].isin(["Residencial", 
																																			   "No Residencial BT",
																																			   "No Residencial AT"]),
																																				ahorro_por_compensacion_evitada_parcializado, 0)

# Ahorro global anual por reducción en la duración promedio de las fallas
print(f"Considera la reducción de un {reduccion_duracion_falla * 100}% en la duración promedio de una falla")
print("------------------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	beneficio_neto_compensacion_evitada = lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"].sum()
	print("Ahorro por compensación evitada periodo {}: M USD {:,.4f}"
		  .format(periodo, beneficio_neto_compensacion_evitada / 1_000_000))

### Beneficio 5: Ahorro por efecto de una reducción del precio de la energía en bloques horarios de suministro

El informe de GTD calcula el beneficion sobre “Gestión de Consumo” suponiendo que, dentro del universo regulado, al 25% de los clientes de mayor consumo (que concentran el 73,92% de la energía vendida) les conviene migrar a una opción horaria y que, gracias al SMI, ese grupo podrá desplazar el 11% de su consumo desde horas punta (Bloque C) a intermedias (Bloque B), valorando ese desplazamiento con un diferencial de 20 USD/MWh entre ambos bloques. Así, el beneficio anual se obtiene multiplicando la energía vendida total del año base del estudio por 0,7392 × 0,11 × 20.

Considerando el año del estudio (2016), se ha ajustado el diferencial de precio por CPI, equivalente a un factor de 1,34.

In [ ]:
# # Ahorro por gestión del consumo energético
# energia_clientes_mayor_consumo = 0.7392
# energia_trasladada = energia_clientes_mayor_consumo * 0.11

# precio_diferencial = 20	* 1.34			# USD/MWh (ajustado por CPI, 2016-2025)

# for periodo in range(len(lista_dfs_implementacion)):
	
# 	# Indice para aplicar el cálculo solo a los segmentos Residencial y No_Residencial
# 	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
# 						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No_Residencial", 1, 0))

# 	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
# 																					* indice_segmento
# 																					* energia_trasladada
# 																					* precio_diferencial)

# # Ahorro global anual por gestión del consumo energético
# print("Ahorro por gestión del consumo energético.")
# print("------------------------------------------------------------------------")
# for periodo in range(len(lista_dfs_implementacion)):
# 	beneficio_neto_gestion_consumo = lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"].sum()
# 	print("Ahorro por gestión del consumo periodo {}: M USD {:,.4f}"
# 		  .format(periodo, beneficio_neto_gestion_consumo / 1_000_000))

La forma de estimar el efecto que tendría implementar bloques horarios de suministro a un precio energético más barato se realiza considerando cuanta energía, y que consumos de ella es posible trasladar de los bloques caros a los bloques baratos.

Para esto se hace uso de la información dispuesta en el siguiente estudio: [Informe: Usos de la Energía en Chile (2018)](https://www.electricas.cl/wp-content/uploads/2020/07/informe-final-usos-energia-chile-2018.pdf). En el se aproxima la responsabilidad del uso energético proveniente de cada artefacto residencial (página 91, Gráfico 10).

En el mismo informe, se dispone de una gráfica (página 33, Gráfico 3) en el que se detalla el porcentaje de clientes que cuentan con un artefacto/equipo en particular.

En sintesis, los equipos o artefactos que en principio podrían desplazar su uso hacia horas de menor consumo son las siguientes:

 - Lavado de Ropa: 1,6% del total de energía, el 98% de los clientes tiene el equipo.
 - Secado de Ropa: 6,2% del total de energía, el 29,9% de los clientes tiene el equipo.
 - Plancha de Ropa: 2,1% del total de energía, el 69,3% de los clientes tiene el equipo.
 - Aspiradora: 5,0% del total de energía, el 53,4% de los clientes tiene el equipo.
 - Hervidor: 4,0% del total de energía, el 77,9% de los clientes tiene el equipo.

También se requiere conocer la curva de demanda característica del cliente residencial, la cual se obtiene a partir del [Modelo CREST](https://figshare.com/articles/dataset/CREST_Demand_Model_v2_0/2001129). De este modelo se estima la proporción de energía consumida en el bloque más caro respecto del total diario.

En cuanto a la reducción del precio, a modo de referencia se pueden revisar los precios de adjudicación de los diferentes contratos de suministro eléctrico, revisando aquellos puramente provenientes de combustibles fósiles, y aquellos provenientes de energías renovables [Informe: Licitaciones de Suministro](https://icrchile.cl/wp-content/uploads/2023/04/E001-1.pdf). El informe asegura que en promedio, los contratos adjudicados han acumulado una caída de 78% entre 2014 y 2021, lo que se explica en gran medida por las tecnologías de generación que están detrás de dichos contratos.

In [ ]:
# Ahorro por reducción del costo de la energía mediante gestión del consumo
while True:
	reduccion_precio_de_la_energia = float(input("Reducción en el precio de la energía por gestión del consumo [%]: ")) / 100
	if 0 <= reduccion_precio_de_la_energia <= 1:
	    break

bloque_caro_de_energia = 0.5126		# % de la energía consumida que se concentra en el bloque caro (horario 18:00 a 7:59hrs)

energia_lavado_de_ropa = 0.016			# % de la energía consumida que representa al lavado de ropa
energia_secado_de_ropa = 0.062			# % de la energía consumida que representa al secado de ropa
energia_plancha_de_ropa = 0.021			# % de la energía consumida que representa al planchado de ropa
energia_aspiradora = 0.050				# % de la energía consumida que representa al uso de la aspiradora
energia_hervidora_agua = 0.040			# % de la energía consumida que representa al hervido de agua

clientes_con_lavadora = 0.980				# % de los clientes que poseen lavadora
clientes_con_secadora = 0.299				# % de los clientes que poseen secadora
clientes_con_plancha = 0.693				# % de los clientes que poseen plancha
clientes_con_aspiradora = 0.534				# % de los clientes que poseen aspiradora
clientes_con_hervidor = 0.779				# % de los clientes que poseen hervidor eléctrico

energia_trasladada_lavado_de_ropa = energia_lavado_de_ropa * clientes_con_lavadora			# % de la energía trasladada del bloque caro al bloque barato sobre el lavado de ropa
energia_trasladada_secado_de_ropa = energia_secado_de_ropa * clientes_con_secadora			# % de la energía trasladada del bloque caro al bloque barato sobre el secado de ropa
energia_trasladada_plancha_de_ropa = energia_plancha_de_ropa * clientes_con_plancha			# % de la energía trasladada del bloque caro al bloque barato sobre el planchado de ropa
energia_trasladada_aspiradora = energia_aspiradora * clientes_con_aspiradora				# % de la energía trasladada del bloque caro al bloque barato sobre el uso de la aspiradora
energia_trasladada_hervidora_agua = energia_hervidora_agua * clientes_con_hervidor			# % de la energía trasladada del bloque caro al bloque barato sobre el hervido de agua

precio_diferencial = precio_energia_usd_mwh * reduccion_precio_de_la_energia		# USD/MWh

for periodo in range(len(lista_dfs_implementacion)):
	
	# Indice para aplicar el cálculo solo a los segmentos Residencial y No_Residencial
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1, 0)

	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																					* indice_segmento
																					* bloque_caro_de_energia
																					* energia_trasladada_lavado_de_ropa
																					* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_secado_de_ropa
																						* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_plancha_de_ropa
																						* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_aspiradora
																						* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_hervidora_agua
																						* precio_diferencial)
	
# Ahorro global anual por gestión del consumo energético
print(f"Ahorro por reducción del precio de la energía en un {reduccion_precio_de_la_energia * 100}%.")
print("------------------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	beneficio_neto_gestion_consumo = lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"].sum()
	print("Ahorro por gestión del consumo periodo {}: M USD {:,.4f}"
		  .format(periodo, beneficio_neto_gestion_consumo / 1_000_000))

### Beneficio 6: Ahorro por efecto de reducción de la cuenta a pagar de energía dada la reducción del consumo de los clientes finales

En cuanto a este tipo de beneficio, se puede contabilizar a través del ahorro generado en la cuenta final de los clientes producto de su reducción en el consumo eléctrico. Este fenómeno se ha estudiado bastante, ejemplos concretos se decriben en [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf), en el cual afirman que:

 - La *European Smart Metering Industry Group* (ESMIG), realizó pilotos de pruebas en los cuales se encontró una reducción promedio del consumo en torno a un 5,4%. Mientras que con feedback del consumo en tiempo real se redujo en promedio un 7,9%.
 - Otros estudios como el de *Energy Demand Research Project* (EDRP) estimaron una reducción en torno al 3% en promedio.
 - Finalmente, en el estudio de despligue de UK se quedan con una reducción promedio del 3% (para es caso residencial) y del 2,8% (para es caso no residencial), aclarando que estos porcentajes podrían subir con el tiempo a medida que la tecnología va madurando.

Para estimar el ahorro que supondría, se tomará como precio de la energía el PNP equivalente nacional calculado en el proceso PNP Julio 2025 ([Anexos PNP Julio 2025](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FPNP%2F2025%2D02%2F02%20ITD%2F4%2E%2DModelo&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)).

In [ ]:
# Estimación del ahorro anual por reducción del consumo
while True:
	reduccion_consumo_residencial = float(input("Reducción del consumo de clientes Residenciales [%]: ")) / 100
	reduccion_consumo_no_residencial = float(input("Reducción del consumo de clientes No Residenciales [%]: ")) / 100
	if (0 <= reduccion_consumo_residencial <= 1 and 0 <= reduccion_consumo_no_residencial <= 1):
		break

clientes_afectos_a_reduccion = 0.075		# % de clientes que reducen su consumo energético (7.5%)

for periodo in range(len(lista_dfs_implementacion)):

	# reducción anual según segmento: Residendial y No Residencial
	reduccion_consumo = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", reduccion_consumo_residencial,
							np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", reduccion_consumo_no_residencial,
								np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", reduccion_consumo_no_residencial, 0)))

	# Se calcula Energía facturada anual [MWh/año] * reducción consumo [%] * precio energía [USD/MWh]
	lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																					* clientes_afectos_a_reduccion
																					* reduccion_consumo 
																					* precio_energia_usd_mwh)

print("Considera la reducción de un:", 
	  f"\n -> {reduccion_consumo_residencial * 100}% en el consumo de energía Residencial",
	  f"\n -> {reduccion_consumo_no_residencial * 100}% en el consumo de energía No Residencial")
print("----------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	beneficio_neto_reduccion_consumo = lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"].sum()
	print("Ahorro por reducción del consumo periodo {}: M USD {:,.4f}"
		  .format(periodo, beneficio_neto_reduccion_consumo / 1_000_000))

### Beneficio 7: Ahorro por efecto de reducción de emisiones dada la reducción del consumo de los clientes finales

Asociado con la reducción del consumo de los clientes finales, esto tiene consecuencia directa en la reducción de emisiones producto de la reducción de unidades de energía eléctrica generadas.

Este beneficio entonces puede ser calculado a través de datos de Ministerio del Medio Ambiente para conocer la euivalencia entre energía generada y emisiones producidas, [Estudio: Emisiones por kWh](https://huellachile.mma.gob.cl/wp-content/uploads/2024/11/HuellaChile-DCC-Factores-de-emision-nivel-basico_v3.pdf).

Además se puede hacer uso del costo social de emitir unidades de CO2 publicado por el Ministerio de Desarrollo Social, [Costo Social de CO2](https://repositorio.cepal.org/server/api/core/bitstreams/cdeda1a0-2d93-4031-a4df-94e2d25b219a/content), en el mismo documento se proyecta el precio hasta el año 2025.

In [ ]:
# Ahorro nacional anual por reducción de emisiones de CO2

# Toneladas de CO2 emitidas por MWh generado (Fuente: Estudio Ministerio del Medio Ambiente, Ver 'Proyecciones_y_Estimaciones.xlsx')
emisiones_co2_por_mwh = [0.2190, 0.2084, 0.1982, 0.1885, 0.1793, 0.1706, 0.1622, 0.1543,
    					 0.1468, 0.1396, 0.1328, 0.1263, 0.1202, 0.1143, 0.1087, 0.1034]

# Estimaciones a partir del informe "Estimación del precio social del carbono para la evaluación de la inversión pública en Chile" (Ver 'Proyecciones_y_Estimaciones.xlsx')
costo_social_co2 = [71.11, 78.83, 86.55, 94.27, 102.00, 109.72, 117.44, 125.16, 
		   			132.88, 140.60, 148.33, 156.05, 163.77, 171.49, 179.21, 186.93]		# Costo social USD/tCO2 (Fuente: Ministerio de Desarrollo Social) desde 2025 a 2040

for periodo in range(len(lista_dfs_implementacion)):

	# reducción consumo anual según segmento: Residendial y No_Residencial
	reduccion_consumo = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", reduccion_consumo_residencial,
							np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", reduccion_consumo_no_residencial,
								np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", reduccion_consumo_no_residencial, 0)))
	
	lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"] = ((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																				  		* clientes_afectos_a_reduccion
																						* reduccion_consumo
																						* emisiones_co2_por_mwh[periodo]
																						* costo_social_co2[periodo])
																					+ ((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																						* hurto_reducido
																						* emisiones_co2_por_mwh[periodo]
																						* costo_social_co2[periodo])
																					+ (((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																						* comercial_reducido
																						* emisiones_co2_por_mwh[periodo]
																						* costo_social_co2[periodo])))

print("Considera la reducción de un:", 
	  f"\n -> {reduccion_consumo_residencial * 100}% en el consumo de energía Residencial",
	  f"\n -> {reduccion_consumo_no_residencial * 100}% en el consumo de energía No Residencial")
print("--------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	beneficio_neto_reduccion_emisiones = lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"].sum()
	print("Ahorro por reducción de emisiones CO2 periodo {}: M USD {:,.4f}"
		  .format(periodo, beneficio_neto_reduccion_emisiones / 1_000_000))

### Beneficio 8: Ahorro por efecto de la reducción del uso de la atención al cliente de las empresas distribuidoras

Este beneficio se obtiene gracias a las reducciones del servicio de atención al cliente debido a la reducción de las estimaciones de consumo, precisión del medidor inteligente, detección automática de fallas e incidencias, mayor información relativa a su consumo personal, etc. Este beneficio se estima, de parte de [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf), en una reducción de las llamadas y comunicaciones con el servicio en torno a un 60%.

Para construir este beneficio para el caso nacional, se hace uso de los resultados dispuestos en [Anexo 4-1, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FVAD%2FVAD%202024%2D2028%2F02%20Informe%20T%C3%A9cnico%20Preliminar%2FATD1%2FAnexos%2FAnexo%204%2FAnexo%204%2D1&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1). Del cual se puede obtener un costo promedio nacional de la atención por cliente al año.

In [ ]:
# Ahorros asociados al corte y reposición remotos de suministro
while True:
	reduccion_atencion_clientes = float(input("Reducción en el uso de la atención al cliente [%]: ")) / 100
	if 0 <= reduccion_atencion_clientes <= 1:
	    break

# Costo promedio de atención al cliente (Fuente: VAD 2024-2028)
atencion_al_cliente = [2.4662, 2.4623, 2.4576, 2.4540, 2.4494, 2.4440, 2.4387, 2.4335,
    				   2.4283, 2.4232, 2.4182, 2.4133, 2.4084, 2.4049, 2.4002, 2.3954]			# USD/cliente-año

for periodo in range(len(lista_dfs_implementacion)):

	# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", 1,
			   				np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", 1, 0)))

	lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"] = (lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																					* atencion_al_cliente[periodo]
																					* reduccion_atencion_clientes
																					* indice_segmento)

# Ahorro global anual por reducción en la atención al cliente
print(f"Considera la reducción de un {reduccion_atencion_clientes * 100}% en el uso de atención al cliente")
print("--------------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Ahorro por reducción en atención al cliente periodo {}: M USD {:,.4f}"
		  .format(periodo, lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"].sum() / 1_000_000))

### Beneficio 9: Ahorro por efecto de la reducción en los requerimientos de inversión de red

Este beneficio tiene que ver con los ahorros generados por la reducción de requerimientos de inversión. Esto se dá gracias a la mejora en la visualización de las redes y el monitoreo de estas, con esta tecnología se puede saber con mucha más precisión las inversiones de red que realmente son necesarias.

Para calcular el beneficio se hace uso de las tablas de inversión de red dispuestas en el [Anexo 5, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/:x:/r/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/Doc.aspx?sourcedoc=%7BCFFC61F6-6AC0-40D6-B74E-3191D176ED1C%7D&file=Anexo%205-VAD%20ATD12.xlsx&action=default&mobileredirect=true). Con ellas se obtiene un monto neto nacional de inversión asociada a cumplir los requerimientos de calidad de la NTD.

[Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf) considera una reducción de entre el 5% y el 10% en este tipo de beneficio.

In [ ]:
# Ahorros sobre la inversión evitada en infraestructura de red
while True:
	reduccion_infraestructura = float(input("Reducción en la inversión de infraestructura de red [%]: ")) / 100
	if 0 <= reduccion_infraestructura <= 1:
	    break

# Costo anualizado nacional de inversión en infraestructura de red (Fuente: VAD 2024-2028)
inversion_infraestructura = [199701412, 199847572, 199997331, 202194814, 202666824, 202973599,
    						203193675, 203407866, 205082472, 205512944, 205851889, 206191313, 206593527, 
							207198255, 207804753, 208413026]     # USD/año

for periodo in range(len(lista_dfs_implementacion)):

	# Inversión evitada
	inversion_evitada = inversion_infraestructura[periodo] * reduccion_infraestructura

	# Se asigna una parte a cada comuna según su "participación" en la inversión total
	ahorro_por_inversion_evitada_parcializada = inversion_evitada / (len(lista_dfs_implementacion[periodo]) / 5)
	lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"] = np.where(lista_dfs_implementacion[periodo]["Segmento"].isin(["Red"]),
																					ahorro_por_inversion_evitada_parcializada, 0)

# Ahorro global anual por reducción en la inversión de infraestructura de red
print(f"Considera la reducción de un {reduccion_infraestructura * 100}% en la inversión de infraestructura de red")
print("---------------------------------------------------------------------------")
for periodo in range(len(lista_dfs_implementacion)):
	print("Ahorro por inversión evitada periodo {}: M USD {:,.4f}"
		  .format(periodo, lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"].sum() / 1_000_000))

### Beneficio Neto

In [ ]:
df_resultado_final_beneficios = pd.DataFrame()

beneficios = [
	"Ahorro en lectura pedestre [USD/año]",
	"Ahorro corte y reposición remoto [USD/año]",
	"Ahorro reducción de PNT [USD/año]",
	"Ahorro por compensación evitada [USD/año]",
	"Ahorro por gestión del consumo [USD/año]",
	"Ahorro reducción del consumo [USD/año]",
	"Ahorro reducción emisiones CO2 [USD/año]",
	"Ahorro atención al cliente [USD/año]",
	"Ahorro por inversión evitada [USD/año]"
]

df_resultado_final_beneficios["Concepto"] = beneficios

for periodo in range(len(lista_dfs_implementacion)):

	nuevo_periodo_beneficios = [
		lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"].sum()
	]

	df_resultado_final_beneficios["Periodo {}".format(periodo)] = nuevo_periodo_beneficios

	# Suma de todos los beneficios anuales
	lista_dfs_implementacion[periodo]["Beneficio total anual [USD/año]"] = (lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"]				# Beneficio 1
																			+ lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"]		# Beneficio 2
																			+ lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"]				# Beneficio 3
																			+ lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"]		# Beneficio 4
																			+ lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"]			# Beneficio 5
																			+ lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"]			# Beneficio 6
																			+ lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"]			# Beneficio 7
																			+ lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"]				# Beneficio 8
																			+ lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"])			# Beneficio 9

	print("Beneficio total anual del proyecto periodo {}: M USD {:,.4f}"
	  		.format(periodo, lista_dfs_implementacion[periodo]["Beneficio total anual [USD/año]"].sum() / 1_000_000))
	
df_resultado_final = pd.concat([df_resultado_final, df_resultado_final_beneficios], ignore_index=True)

### Cálculos de acuerdo a los requerimientos en cantidad de datos por densidad

In [ ]:
### Capacidad mensual según tecnología de comunicaciones

# Se asignan los resultados del primer período para análisis requerimientos de datos
df_requerimientos = df_implementacion.copy()

# Se rellenan los valores 0 de MB Mensuales por DCU con 1 (¡Esto es sólo para considerar los datos de todas las comunas, no tiene impacto en el análisis económico)
df_requerimientos.loc[((df_requerimientos["Cantidad DCU"] == 0) & (df_requerimientos["Segmento"] == "Red")), "Cantidad DCU"] = 1

# Se calcula la cantidad de datos transmitida por DCU (Cálulos realizados en 'Costos.xlsx')

# 1: Se consideran las exigencias de medición de la Norma SMMC hasta antes del 2034 (Artículo 9-8)
# 0: Se consideran las exigencias de medición de la Norma SMMC desde el 2034 en adelante (Artículo 9-8)
exigencias_de_medicion = 0		# Input: 1 o 0

if exigencias_de_medicion == 1:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 45.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 32.5 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 45.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 38.5 / 1024		# MB mensual por trafo

else:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 71.7 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 83.7 / 1024		# MB mensual por trafo

valores = []

for i, row in df_requerimientos.iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
	cantidad_dcu = df_requerimientos.loc[i, "Cantidad DCU"]

	# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
	if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA"):
		total_clientes_3f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
		total_clientes_1f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

		total_trafo_3f_zona_alta = (df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
									+ df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
		total_trafo_1f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

		mb_exigidos = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
								+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
		valores.append(mb_exigidos)

	elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA"):
		total_clientes_3f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
		total_clientes_1f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

		total_trafo_3f_zona_baja = (df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
									+ df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
		total_trafo_1f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

		mb_exigidos = (datos_cliente_3f_zona_baja * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja * total_clientes_1f_zona_baja
								+ datos_trafo_3f_zona_baja * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja * total_trafo_1f_zona_baja) / cantidad_dcu
		valores.append(mb_exigidos)

	else:        
		mb_exigidos = 0								# Caso de Celular, TWACS y otros segmentos
		valores.append(mb_exigidos)
		
df_requerimientos["MB Mensuales por DCU"] = 0.0
filtro = df_requerimientos["Segmento"] == "Red"

df_requerimientos.loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=df_requerimientos.index[filtro]).round(4)


### Capacidad mensual según tecnología de comunicaciones
capacidad_twacs_por_dcu = ((300 / 8) * 60 * 60 * 24 * 30) / 1_000_000				# 300bps a 97,2MB mensuales por DCU TWACS
capacidad_g3plc_por_dcu = ((35 * 1000 / 8) * 60 * 60 * 24 * 30) / 1_000_000			# 35kbps a 11.340MB mensuales por DCU G3-PLC


# Variables auxiliares para contabilizar las comunas con capacidad suficiente
comunas_con_capacidad_suficiente_alta = 0
comunas_con_capacidad_suficiente_media = 0
comunas_con_capacidad_suficiente_baja = 0
comunas_con_capacidad_suficiente_muy_baja = 0
comunas_con_capacidad_suficiente_extremadamente_baja = 0

# Se recorre la fila "red" de cada comuna
for i, row in df_requerimientos.iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"
	densidad = row["Densidad"]
	mb_mensuales = row["MB Mensuales por DCU"]
	tecnologia = row["Tecnología"]

	if densidad == "ALTA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_alta += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_alta += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
	
	elif densidad == "MEDIA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_media += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_media += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
	
	elif densidad == "BAJA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_baja += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_baja += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
	
	elif densidad == "MUY BAJA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_muy_baja += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_muy_baja += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
	
	else:	# EXTREMADAMENTE BAJA
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_extremadamente_baja += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_extremadamente_baja += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
	
total_comunas_alta = len(df_requerimientos[df_requerimientos['Densidad'] == 'ALTA']) // 5
total_comunas_media = len(df_requerimientos[df_requerimientos['Densidad'] == 'MEDIA']) // 5
total_comunas_baja = len(df_requerimientos[df_requerimientos['Densidad'] == 'BAJA']) // 5
total_comunas_muy_baja = len(df_requerimientos[df_requerimientos['Densidad'] == 'MUY BAJA']) // 5
total_comunas_extremadamente_baja = len(df_requerimientos[df_requerimientos['Densidad'] == 'EXTREMADAMENTE BAJA']) // 5

# Resultados de capacidad suficiente
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")
print("----------------------------------------------------------------------")
print("Capacidad mensual según tecnología de comunicaciones:")
print(f" - Comunas con densidad ALTA y capacidad suficiente: {comunas_con_capacidad_suficiente_alta} de {total_comunas_alta}, ({(comunas_con_capacidad_suficiente_alta / total_comunas_alta) * 100:.2f}%)")
print(f" - Comunas con densidad MEDIA y capacidad suficiente: {comunas_con_capacidad_suficiente_media} de {total_comunas_media}, ({(comunas_con_capacidad_suficiente_media / total_comunas_media) * 100:.2f}%)")
print(f" - Comunas con densidad BAJA y capacidad suficiente: {comunas_con_capacidad_suficiente_baja} de {total_comunas_baja}, ({(comunas_con_capacidad_suficiente_baja / total_comunas_baja) * 100:.2f}%)")
print(f" - Comunas con densidad MUY BAJA y capacidad suficiente: {comunas_con_capacidad_suficiente_muy_baja} de {total_comunas_muy_baja}, ({(comunas_con_capacidad_suficiente_muy_baja / total_comunas_muy_baja) * 100:.2f}%)")
print(f" - Comunas con densidad EXTREMADAMENTE BAJA y capacidad suficiente: {comunas_con_capacidad_suficiente_extremadamente_baja} de {total_comunas_extremadamente_baja}, ({(comunas_con_capacidad_suficiente_extremadamente_baja / 
																																											 total_comunas_extremadamente_baja) * 100:.2f}%)")

### Cálculo sobre la inversión y operación instantáneos por densidad

In [ ]:
### Calculo sobre los costos netos de inversión y operación por tipo de densidad
costos_por_densidad = df_requerimientos.groupby(["Tecnología", "Densidad"]).agg({
	"Inversión Total [USD]": "sum",
	"Costos Operacionales Totales [USD/año]": "sum"
}).reset_index()

for i, row in costos_por_densidad.iterrows():
	densidad = row["Densidad"]
	inversion_total = row["Inversión Total [USD]"]
	costos_operacionales = row["Costos Operacionales Totales [USD/año]"]

	if densidad == "ALTA":
		tecnología_alta = row["Tecnología"]
		inversion_promedio_alta = inversion_total / total_comunas_alta / 1_000_000		# En millones de USD
		costos_operacionales_promedio_alta = costos_operacionales / total_comunas_alta / 1_000_000
	
	elif densidad == "MEDIA":
		tecnología_media = row["Tecnología"]
		inversion_promedio_media = inversion_total / total_comunas_media / 1_000_000		# En millones de USD
		costos_operacionales_promedio_media = costos_operacionales / total_comunas_media / 1_000_000	
	
	elif densidad == "BAJA":
		tecnología_baja = row["Tecnología"]
		inversion_promedio_baja = inversion_total / total_comunas_baja / 1_000_000		# En millones de USD
		costos_operacionales_promedio_baja = costos_operacionales / total_comunas_baja / 1_000_000
		
	elif densidad == "MUY BAJA":
		tecnología_muy_baja = row["Tecnología"]
		inversion_promedio_muy_baja = inversion_total / total_comunas_muy_baja / 1_000_000		# En millones de USD
		costos_operacionales_promedio_muy_baja = costos_operacionales / total_comunas_muy_baja / 1_000_000
		
	else:	# EXTREMADAMENTE BAJA
		tecnología_extremadamente_baja = row["Tecnología"]
		inversion_promedio_extremadamente_baja = inversion_total / total_comunas_extremadamente_baja / 1_000_000		# En millones de USD
		costos_operacionales_promedio_extremadamente_baja = costos_operacionales / total_comunas_extremadamente_baja / 1_000_000

print("Costos netos promedio por tipo de densidad:")
print("----------------------------------------------------------------------")
print(f"Tecnología densidad ALTA: {tecnología_alta} | Inversión promedio [MUSD] {inversion_promedio_alta:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_alta:,.2f}")
print(f"Tecnología densidad MEDIA: {tecnología_media} | Inversión promedio [MUSD] {inversion_promedio_media:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_media:,.2f}")
print(f"Tecnología densidad BAJA: {tecnología_baja} | Inversión promedio [MUSD] {inversion_promedio_baja:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_baja:,.2f}")
print(f"Tecnología densidad MUY BAJA: {tecnología_muy_baja} | Inversión promedio [MUSD] {inversion_promedio_muy_baja:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_muy_baja:,.2f}")
print(f"Tecnología densidad EXTREMADAMENTE BAJA: {tecnología_extremadamente_baja} | Inversión promedio [MUSD] {inversion_promedio_extremadamente_baja:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_extremadamente_baja:,.2f}")

### Cálculo sobre la cantidad de brigadas requeridas para un objetivo de implementación dado (caso base = 10 años)

In [ ]:
### Sobre el cálculo de la cantidad de brigadas necesarias para la instalación de UM
# Se declaran las productividades diarias promedio en función de la densidad y el tipo de equipo
prod_um1f_alta = 10			# UM/día en zona alta
prod_um3fd_alta = 8			# UM/día en zona alta
prod_um3fi_alta = 3			# UM/día en zona alta

prod_um1f_media = 8			# UM/día en zona media
prod_um3fd_media = 6		# UM/día en zona media
prod_um3fi_media = 3		# UM/día en zona media

prod_um1f_baja = 6			# UM/día en zona baja
prod_um3fd_baja = 4			# UM/día en zona baja
prod_um3fi_baja = 2			# UM/día en zona baja

prod_um1f_muy_baja = 4		# UM/día en zona muy baja
prod_um3fd_muy_baja = 3		# UM/día en zona muy baja	
prod_um3fi_muy_baja = 2		# UM/día en zona muy baja

prod_um1f_extremadamente_baja = 3		# UM/día en zona extremadamente baja
prod_um3fd_extremadamente_baja = 2		# UM/día en zona extremadamente baja	
prod_um3fi_extremadamente_baja = 1		# UM/día en zona extremadamente baja

objetivo_del_despliegue = 10		# Años para completar el despliegue

brigadas_zona_alta = 0
brigadas_zona_media = 0
brigadas_zona_baja = 0
brigadas_zona_muy_baja = 0
brigadas_zona_extremadamente_baja = 0

for i, row in df_implementacion.iterrows():
	densidad = row["Densidad"]
	um1f = row["Cantidad UM 1F"] / (objetivo_del_despliegue * 260)
	um3fd = row["Cantidad UM 3F Directa"] / (objetivo_del_despliegue * 260)
	um3fi = row["Cantidad UM 3F Indirecta"] / (objetivo_del_despliegue * 260)
	
	if densidad == "ALTA":
		brigadas_um1f = um1f / prod_um1f_alta
		brigadas_um3fd = um3fd / prod_um3fd_alta
		brigadas_um3fi = um3fi / prod_um3fi_alta
		brigadas_zona_alta += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	elif densidad == "MEDIA":
		brigadas_um1f = um1f / prod_um1f_media
		brigadas_um3fd = um3fd / prod_um3fd_media
		brigadas_um3fi = um3fi / prod_um3fi_media
		brigadas_zona_media += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	elif densidad == "BAJA":
		brigadas_um1f = um1f / prod_um1f_baja
		brigadas_um3fd = um3fd / prod_um3fd_baja
		brigadas_um3fi = um3fi / prod_um3fi_baja
		brigadas_zona_baja += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	elif densidad == "MUY BAJA":
		brigadas_um1f = um1f / prod_um1f_muy_baja
		brigadas_um3fd = um3fd / prod_um3fd_muy_baja
		brigadas_um3fi = um3fi / prod_um3fi_muy_baja
		brigadas_zona_muy_baja += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	else:	# EXTREMADAMENTE BAJA
		brigadas_um1f = um1f / prod_um1f_extremadamente_baja
		brigadas_um3fd = um3fd / prod_um3fd_extremadamente_baja
		brigadas_um3fi = um3fi / prod_um3fi_extremadamente_baja
		brigadas_zona_extremadamente_baja += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

total_brigadas = (brigadas_zona_alta + brigadas_zona_media + brigadas_zona_baja
					+ brigadas_zona_muy_baja + brigadas_zona_extremadamente_baja)

print("Cantidad de brigadas necesarias para la instalación de UM en un plazo de {} años: {:,.2f} brigadas".format(objetivo_del_despliegue, total_brigadas))